In [22]:
import sqlite3
import pandas as pd
pd.set_option('max_columns', 180)
pd.set_option('max_rows', 200000)
pd.set_option('max_colwidth', 5000)

In [23]:
glog = pd.read_csv('game_log.csv', low_memory = False)

In [24]:
#read csv's, print shap
pacodes = pd.read_csv('park_codes.csv')
pecodes = pd.read_csv('person_codes.csv')
tcodes = pd.read_csv('team_codes.csv')

print('games_log: ',glog.shape,'\n','park_codes: ',pacodes.shape)
print('person_codes: ',pecodes.shape)
print('team_codes: ',tcodes.shape)

games_log:  (171907, 161) 
 park_codes:  (252, 9)
person_codes:  (20494, 7)
team_codes:  (150, 8)


In [ ]:
#game_log fields explaination
!cat game_log_fields.txt

In [5]:
#game_log columns
glog.head(1)

,date,number_of_game,day_of_week,v_name,v_league,v_game_number,h_name,h_league,h_game_number,v_score,h_score,length_outs,day_night,completion,forefeit,protest,park_id,attendance,length_minutes,v_line_score,h_line_score,v_at_bats,v_hits,v_doubles,v_triples,v_homeruns,v_rbi,v_sacrifice_hits,v_sacrifice_flies,v_hit_by_pitch,v_walks,v_intentional_walks,v_strikeouts,v_stolen_bases,v_caught_stealing,v_grounded_into_double,v_first_catcher_interference,v_left_on_base,v_pitchers_used,v_individual_earned_runs,v_team_earned_runs,v_wild_pitches,v_balks,v_putouts,v_assists,v_errors,v_passed_balls,v_double_plays,v_triple_plays,h_at_bats,h_hits,h_doubles,h_triples,h_homeruns,h_rbi,h_sacrifice_hits,h_sacrifice_flies,h_hit_by_pitch,h_walks,h_intentional_walks,h_strikeouts,h_stolen_bases,h_caught_stealing,h_grounded_into_double,h_first_catcher_interference,h_left_on_base,h_pitchers_used,h_individual_earned_runs,h_team_earned_runs,h_wild_pitches,h_balks,h_putouts,h_assists,h_errors,h_passed_balls,h_double_plays,h_triple_plays,hp_umpire_id,hp_umpire_name,1b_umpire_id,1b_umpire_name,2b_umpire_id,2b_umpire_name,3b_umpire_id,3b_umpire_name,lf_umpire_id,lf_umpire_name,rf_umpire_id,rf_umpire_name,v_manager_id,v_manager_name,h_manager_id,h_manager_name,winning_pitcher_id,winning_pitcher_name,losing_pitcher_id,losing_pitcher_name,saving_pitcher_id,saving_pitcher_name,winning_rbi_batter_id,winning_rbi_batter_id_name,v_starting_pitcher_id,v_starting_pitcher_name,h_starting_pitcher_id,h_starting_pitcher_name,v_player_1_id,v_player_1_name,v_player_1_def_pos,v_player_2_id,v_player_2_name,v_player_2_def_pos,v_player_3_id,v_player_3_name,v_player_3_def_pos,v_player_4_id,v_player_4_name,v_player_4_def_pos,v_player_5_id,v_player_5_name,v_player_5_def_pos,v_player_6_id,v_player_6_name,v_player_6_def_pos,v_player_7_id,v_player_7_name,v_player_7_def_pos,v_player_8_id,v_player_8_name,v_player_8_def_pos,v_player_9_id,v_player_9_name,v_player_9_def_pos,h_player_1_id,h_player_1_name,h_player_1_def_pos,h_player_2_id,h_player_2_name,h_player_2_def_pos,h_player_3_id,h_player_3_name,h_player_3_def_pos,h_player_4_id,h_player_4_name,h_player_4_def_pos,h_player_5_id,h_player_5_name,h_player_5_def_pos,h_player_6_id,h_player_6_name,h_player_6_def_pos,h_player_7_id,h_player_7_name,h_player_7_def_pos,h_player_8_id,h_player_8_name,h_player_8_def_pos,h_player_9_id,h_player_9_name,h_player_9_def_pos,additional_info,acquisition_info
0,18710504,0,Thu,CL1,NaN,1,FW1,NaN,1,0,2,54.0,D,NaN,NaN,NaN,FOR01,200.0,120.0,000000000,010010000,30.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,6.0,1.0,NaN,-1.0,NaN,4.0,1.0,1.0,1.0,0.0,0.0,27.0,9.0,0.0,3.0,0.0,0.0,31.0,4.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,NaN,0.0,0.0,NaN,-1.0,NaN,3.0,1.0,0.0,0.0,0.0,0.0,27.0,3.0,3.0,1.0,1.0,0.0,boakj901,John Boake,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,paboc101,Charlie Pabor,lennb101,Bill Lennon,mathb101,Bobby Mathews,prata101,Al Pratt,NaN,NaN,NaN,NaN,prata101,Al Pratt,mathb101,Bobby Mathews,whitd102,Deacon White,2.0,kimbg101,Gene Kimball,4.0,paboc101,Charlie Pabor,7.0,allia101,Art Allison,8.0,white104,Elmer White,9.0,prata101,Al Pratt,1.0,sutte101,Ezra Sutton,5.0,carlj102,Jim Carleton,3.0,bassj101,John Bass,6.0,selmf101,Frank Sellman,5.0,mathb101,Bobby Mathews,1.0,foraj101,Jim Foran,3.0,goldw101,Wally Goldsmith,6.0,lennb101,Bill Lennon,2.0,caret101,Tom Carey,4.0,mince101,Ed Mincher,7.0,mcdej101,James McDermott,8.0,kellb105,Bill Kelly,9.0,NaN,Y


<font color=blue>NOTE</font>

Defensive position numbers:

- `player_1_def_pos` is pitcher, `2` catcher, `3` 1st baseman, `4` 2nd baseman, `5` 3rd baseman, `6` shortstop, `7` left field, `8` center field, `9`right field

In [6]:
#park_codes columns
pacodes.head(1)

,park_id,name,aka,city,state,start,end,league,notes
0,ALB01,Riverside Park,NaN,Albany,NY,09/11/1880,05/30/1882,NL,TRN:9/11/80;6/15&9/10/1881;5/16-5/18&5/30/1882


<font color=blue>NOTE</font>

This table gives information on ball parks used in MLB.

For the database table keys the `park_codes` park_id same as `games_log` park_id

In [7]:
#player, manager, coach, ump information
pecodes.head(3)

,id,last,first,player_debut,mgr_debut,coach_debut,ump_debut
0,aardd001,Aardsma,David,04/06/2004,NaN,NaN,NaN
1,aaroh101,Aaron,Hank,04/13/1954,NaN,NaN,NaN
2,aarot101,Aaron,Tommie,04/10/1962,NaN,04/06/1979,NaN


<font color=blue>NOTE</font>

This table gives information on players, managers, coaches and umpires when they debuted in MLB.

For the database keys `winning pitcher id`, `losing pitcher id`, `umpire id` (home, 1st,2nd,3rd,lf,rf), `v manager and coach`,`h manager and coach` and `player` from the game log table connects to `id` in this table.

In [8]:
#team information
tcodes.head(3)

,team_id,league,start,end,city,nickname,franch_id,seq
0,ALT,UA,1884,1884,Altoona,Mountain Cities,ALT,1
1,ARI,NL,1998,0,Arizona,Diamondbacks,ARI,1
2,BFN,NL,1879,1885,Buffalo,Bisons,BFN,1


In [9]:
tcodes.league.value_counts()

NL    45
AL    25
AA    24
UA    13
FL     9
PL     8
Name: league, dtype: int64

In [10]:
# What is the FL league?  Found info on the others
tcodes.loc[tcodes['league'] == 'FL'].head(2)

,team_id,league,start,end,city,nickname,franch_id,seq
10,BLF,FL,1914,1915,Baltimore,Terrapins,BLF,1
19,BRF,FL,1914,1915,Brooklyn,Tip-Tops,BRF,1


<font color=blue>NOTE</font>

the *tcodes* table gives information on the teams in MLB:

- `team_id` and `franchise_id` identify the team
- `league` gives the league (AL/NL- Current American and National Leagues, AA - American Association (late 1800's), PL - Players League [1890], UA - Union Association [1884], FL - Federal League (1913- 1915)
- `start` and `end` when the team originated and ended

For the database keys `v/h team league`, `v/h team name` as `league` and `team_id`

In [26]:
conn = sqlite3.connect('mlb.db')

In [27]:
#sqlite3 query and command helper functions

def run_query(rq):
    with sqlite3.connect ('mlb.db') as conn:
        return pd.read_sql(rq,conn)

def run_command(rc):
    with sqlite3.connect ('mlb.db') as conn:
        conn.isolation_level = None
        conn.execute('PRAGMA foreign_keys = ON;')
        conn.execute(rc)

In [28]:
#show current database tables
def show_tables():
    q = '''
    SELECT
        name,
        type
    FROM sqlite_master
    WHERE type IN ("table","view");
    '''
    return run_query(q)

In [29]:
show_tables()

,name,type
0,team_appearance,table


In [30]:
#convert df's to sql tables
glog.to_sql('game_log', conn, index=False)

In [31]:
pacodes.to_sql('park_codes', conn, index = False)
pecodes.to_sql('person_codes', conn, index = False)
tcodes.to_sql('team_codes', conn, index = False)

In [32]:
#new game_log column for game_id
new_col = '''
ALTER TABLE game_log
ADD COLUMN game_id TEXT;
'''
run_command(new_col)

In [33]:
#data in game_idcolumn
new_col_data = '''

UPDATE game_log
SET game_id = h_name || date || number_of_game;
'''
run_command(new_col_data)

In [34]:
# check game_id column correct
check_glupdate = '''

SELECT
    game_id,
    h_name,
    date,
    number_of_game
FROM game_log
LIMIT 1;

'''

run_query(check_glupdate)

,game_id,h_name,date,number_of_game
0,FW1187105040,FW1,18710504,0


In [35]:
# Look at game_log SQL table
g_log = '''

SELECT *
FROM game_log
LIMIT 1;'''

run_query(g_log)

,date,number_of_game,day_of_week,v_name,v_league,v_game_number,h_name,h_league,h_game_number,v_score,h_score,length_outs,day_night,completion,forefeit,protest,park_id,attendance,length_minutes,v_line_score,h_line_score,v_at_bats,v_hits,v_doubles,v_triples,v_homeruns,v_rbi,v_sacrifice_hits,v_sacrifice_flies,v_hit_by_pitch,v_walks,v_intentional_walks,v_strikeouts,v_stolen_bases,v_caught_stealing,v_grounded_into_double,v_first_catcher_interference,v_left_on_base,v_pitchers_used,v_individual_earned_runs,v_team_earned_runs,v_wild_pitches,v_balks,v_putouts,v_assists,v_errors,v_passed_balls,v_double_plays,v_triple_plays,h_at_bats,h_hits,h_doubles,h_triples,h_homeruns,h_rbi,h_sacrifice_hits,h_sacrifice_flies,h_hit_by_pitch,h_walks,h_intentional_walks,h_strikeouts,h_stolen_bases,h_caught_stealing,h_grounded_into_double,h_first_catcher_interference,h_left_on_base,h_pitchers_used,h_individual_earned_runs,h_team_earned_runs,h_wild_pitches,h_balks,h_putouts,h_assists,h_errors,h_passed_balls,h_double_plays,h_triple_plays,hp_umpire_id,hp_umpire_name,1b_umpire_id,1b_umpire_name,2b_umpire_id,2b_umpire_name,3b_umpire_id,3b_umpire_name,lf_umpire_id,lf_umpire_name,rf_umpire_id,rf_umpire_name,v_manager_id,v_manager_name,h_manager_id,h_manager_name,winning_pitcher_id,winning_pitcher_name,losing_pitcher_id,losing_pitcher_name,saving_pitcher_id,saving_pitcher_name,winning_rbi_batter_id,winning_rbi_batter_id_name,v_starting_pitcher_id,v_starting_pitcher_name,h_starting_pitcher_id,h_starting_pitcher_name,v_player_1_id,v_player_1_name,v_player_1_def_pos,v_player_2_id,v_player_2_name,v_player_2_def_pos,v_player_3_id,v_player_3_name,v_player_3_def_pos,v_player_4_id,v_player_4_name,v_player_4_def_pos,v_player_5_id,v_player_5_name,v_player_5_def_pos,v_player_6_id,v_player_6_name,v_player_6_def_pos,v_player_7_id,v_player_7_name,v_player_7_def_pos,v_player_8_id,v_player_8_name,v_player_8_def_pos,v_player_9_id,v_player_9_name,v_player_9_def_pos,h_player_1_id,h_player_1_name,h_player_1_def_pos,h_player_2_id,h_player_2_name,h_player_2_def_pos,h_player_3_id,h_player_3_name,h_player_3_def_pos,h_player_4_id,h_player_4_name,h_player_4_def_pos,h_player_5_id,h_player_5_name,h_player_5_def_pos,h_player_6_id,h_player_6_name,h_player_6_def_pos,h_player_7_id,h_player_7_name,h_player_7_def_pos,h_player_8_id,h_player_8_name,h_player_8_def_pos,h_player_9_id,h_player_9_name,h_player_9_def_pos,additional_info,acquisition_info,game_id
0,18710504,0,Thu,CL1,None,1,FW1,None,1,0,2,54.0,D,None,None,None,FOR01,200.0,120.0,000000000,010010000,30.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,None,6.0,1.0,None,-1.0,None,4.0,1.0,1.0,1.0,0.0,0.0,27.0,9.0,0.0,3.0,0.0,0.0,31.0,4.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,None,0.0,0.0,None,-1.0,None,3.0,1.0,0.0,0.0,0.0,0.0,27.0,3.0,3.0,1.0,1.0,0.0,boakj901,John Boake,None,None,None,None,None,None,None,None,None,None,paboc101,Charlie Pabor,lennb101,Bill Lennon,mathb101,Bobby Mathews,prata101,Al Pratt,None,None,None,None,prata101,Al Pratt,mathb101,Bobby Mathews,whitd102,Deacon White,2.0,kimbg101,Gene Kimball,4.0,paboc101,Charlie Pabor,7.0,allia101,Art Allison,8.0,white104,Elmer White,9.0,prata101,Al Pratt,1.0,sutte101,Ezra Sutton,5.0,carlj102,Jim Carleton,3.0,bassj101,John Bass,6.0,selmf101,Frank Sellman,5.0,mathb101,Bobby Mathews,1.0,foraj101,Jim Foran,3.0,goldw101,Wally Goldsmith,6.0,lennb101,Bill Lennon,2.0,caret101,Tom Carey,4.0,mince101,Ed Mincher,7.0,mcdej101,James McDermott,8.0,kellb105,Bill Kelly,9.0,None,Y,FW1187105040


In [36]:
# Look at person_codes SQL table
pe_code = '''

SELECT *
FROM person_codes
LIMIT 1;'''

run_query(pe_code)

,id,last,first,player_debut,mgr_debut,coach_debut,ump_debut
0,aardd001,Aardsma,David,04/06/2004,None,None,None


In [37]:
# Look at park_codes SQL table
pa_code = '''

SELECT *
FROM park_codes
LIMIT 1;'''

run_query(pa_code)

,park_id,name,aka,city,state,start,end,league,notes
0,ALB01,Riverside Park,None,Albany,NY,09/11/1880,05/30/1882,NL,TRN:9/11/80;6/15&9/10/1881;5/16-5/18&5/30/1882


In [38]:
# Look at team_codes SQL table
t_code = '''

SELECT *
FROM team_codes
LIMIT 1;'''

run_query(t_code)

,team_id,league,start,end,city,nickname,franch_id,seq
0,ALT,UA,1884,1884,Altoona,Mountain Cities,ALT,1


<font color=green>OBSERVATIONS</font>

Looking at the data tables, there may be opportunities to remove unnecessary columns and to join columns.  Also need to identify keys.  Here are some column changes and keys:

*Keys*

- `team_id` from **team_codes** shared with `v_name` and `h_name` from **game_log**

- `park_id` from **park_codes** also in **game_log**

*Other*

- The **person_codes** table seems unnecessary.  All of that information can be determined from the **game_log**.

- Dataquest suggested stacking the players `id`, `name` and `def_pos` columns in **game_log**, then adding a batting_order column.  That makes no sense to me; those player positions are game specific.  All you'd end up doing is shifting columns into an increased number of rows.

<font color=blue>schema for MLB database</font>:




![Schema for MLB database](C:\Users\drrdm\Data Quest Guided Projects\3rd SQL Guided Project - MLB\MLB_schema.png)

**To complete project, must use the schema provided by Dataquest.**

In [39]:
# create person table

create_person = '''

CREATE TABLE IF NOT EXISTS person (
person_id TEXT PRIMARY KEY,
first_name TEXT,
last_name TEXT);
'''
data_person = '''

INSERT OR IGNORE INTO person
SELECT 
    id,
    first,
    last
FROM person_codes;
'''
run_command(create_person)
run_command(data_person)

In [40]:
show_person = '''

SELECT *
FROM person
LIMIT 3;
'''
run_query(show_person)

,person_id,first_name,last_name
0,aardd001,David,Aardsma
1,aaroh101,Hank,Aaron
2,aarot101,Tommie,Aaron


In [41]:
# create league table
drop_league = '''
DROP TABLE IF EXISTS league;
'''

create_league = '''

CREATE TABLE IF NOT EXISTS league (
league_id TEXT PRIMARY KEY,
name TEXT);
'''
data_league = '''

INSERT OR IGNORE INTO league (league_id, name)
VALUES 
    ('NL', 'National League'), 
    ('AL', 'American League'),
    ('AA', 'American Association'), 
    ('UA', 'Union Association'), 
    ('FL', 'Federal League'), 
    ('PL', 'Players League');
'''

show_league = '''
SELECT *
FROM league
'''
run_command(drop_league)
run_command(create_league)
run_command(data_league)
run_query (show_league)

,league_id,name
0,NL,National League
1,AL,American League
2,AA,American Association
3,UA,Union Association
4,FL,Federal League
5,PL,Players League


In [27]:
# create park table
#drop_park = '''
#DROP TABLE IF EXISTS park;
#'''

#run_command(drop_league)

create_park = '''

CREATE TABLE IF NOT EXISTS park (
park_id TEXT PRIMARY KEY,
name TEXT,
nickname TEXT,
city TEXT,
state TEXT,
notes TEXT);
'''
data_park = '''

INSERT OR IGNORE INTO park 
SELECT 
    park_id, 
    name,
    aka,
    city,
    state,
    notes
FROM park_codes;
'''

show_park = '''
SELECT *
FROM park
LIMIT 5;
'''

run_command(create_park)
run_command(data_park)
run_query (show_park)

,park_id,name,nickname,city,state,notes
0,ALB01,Riverside Park,None,Albany,NY,TRN:9/11/80;6/15&9/10/1881;5/16-5/18&5/30/1882
1,ALT01,Columbia Park,None,Altoona,PA,None
2,ANA01,Angel Stadium of Anaheim,Edison Field; Anaheim Stadium,Anaheim,CA,None
3,ARL01,Arlington Stadium,None,Arlington,TX,None
4,ARL02,Rangers Ballpark in Arlington,The Ballpark in Arlington; Ameriquest Fl,Arlington,TX,None


In [42]:
# create appearance_type table
drop_appearance_type = '''
DROP TABLE IF EXISTS appearance_type;
'''
run_command(drop_appearance_type)

create_appearance_type = '''

CREATE TABLE IF NOT EXISTS appearance_type (
appearance_id TEXT PRIMARY KEY,
name TEXT,
category TEXT);
'''

at = pd.read_csv('appearance_type.csv')
with sqlite3.connect ('mlb.db') as conn:
    at.to_sql('appearance_type', conn, index=False)


show_appearance_type = '''
SELECT *
FROM appearance_type
LIMIT 5;
'''

run_command(create_appearance_type)
run_query (show_appearance_type)

,appearance_type_id,name,category
0,O1,Batter 1,offense
1,O2,Batter 2,offense
2,O3,Batter 3,offense
3,O4,Batter 4,offense
4,O5,Batter 5,offense


In [43]:
# Look at team_codes SQL table
t_code = '''

SELECT *
FROM team_codes
LIMIT 1;'''

run_query(t_code)

,team_id,league,start,end,city,nickname,franch_id,seq
0,ALT,UA,1884,1884,Altoona,Mountain Cities,ALT,1


In [44]:
# create team table
#drop_team = '''
#DROP TABLE IF EXISTS team;
#'''

#run_command(drop_team)

create_team = '''

CREATE TABLE IF NOT EXISTS team (
team_id TEXT PRIMARY KEY,
league_id TEXT ,
nickname TEXT,
city TEXT,
franch_id TEXT,
FOREIGN KEY (league_id) REFERENCES league (league_id));
'''
data_team = '''

INSERT OR IGNORE INTO team 
SELECT 
    team_id, 
    league,
    nickname,
    city,
    franch_id
FROM team_codes;
'''

show_team = '''
SELECT *
FROM team
LIMIT 5;
'''

In [45]:
run_command(create_team)
run_command(data_team)
run_query (show_team)

,team_id,league_id,nickname,city,franch_id
0,ALT,UA,Mountain Cities,Altoona,ALT
1,ARI,NL,Diamondbacks,Arizona,ARI
2,BFN,NL,Bisons,Buffalo,BFN
3,BFP,PL,Bisons,Buffalo,BFP
4,BL1,None,Canaries,Baltimore,BL1


In [46]:
# Look at game_log SQL table
g_log = '''

SELECT *
FROM game_log
LIMIT 1;'''

run_query(g_log)

,date,number_of_game,day_of_week,v_name,v_league,v_game_number,h_name,h_league,h_game_number,v_score,h_score,length_outs,day_night,completion,forefeit,protest,park_id,attendance,length_minutes,v_line_score,h_line_score,v_at_bats,v_hits,v_doubles,v_triples,v_homeruns,v_rbi,v_sacrifice_hits,v_sacrifice_flies,v_hit_by_pitch,v_walks,v_intentional_walks,v_strikeouts,v_stolen_bases,v_caught_stealing,v_grounded_into_double,v_first_catcher_interference,v_left_on_base,v_pitchers_used,v_individual_earned_runs,v_team_earned_runs,v_wild_pitches,v_balks,v_putouts,v_assists,v_errors,v_passed_balls,v_double_plays,v_triple_plays,h_at_bats,h_hits,h_doubles,h_triples,h_homeruns,h_rbi,h_sacrifice_hits,h_sacrifice_flies,h_hit_by_pitch,h_walks,h_intentional_walks,h_strikeouts,h_stolen_bases,h_caught_stealing,h_grounded_into_double,h_first_catcher_interference,h_left_on_base,h_pitchers_used,h_individual_earned_runs,h_team_earned_runs,h_wild_pitches,h_balks,h_putouts,h_assists,h_errors,h_passed_balls,h_double_plays,h_triple_plays,hp_umpire_id,hp_umpire_name,1b_umpire_id,1b_umpire_name,2b_umpire_id,2b_umpire_name,3b_umpire_id,3b_umpire_name,lf_umpire_id,lf_umpire_name,rf_umpire_id,rf_umpire_name,v_manager_id,v_manager_name,h_manager_id,h_manager_name,winning_pitcher_id,winning_pitcher_name,losing_pitcher_id,losing_pitcher_name,saving_pitcher_id,saving_pitcher_name,winning_rbi_batter_id,winning_rbi_batter_id_name,v_starting_pitcher_id,v_starting_pitcher_name,h_starting_pitcher_id,h_starting_pitcher_name,v_player_1_id,v_player_1_name,v_player_1_def_pos,v_player_2_id,v_player_2_name,v_player_2_def_pos,v_player_3_id,v_player_3_name,v_player_3_def_pos,v_player_4_id,v_player_4_name,v_player_4_def_pos,v_player_5_id,v_player_5_name,v_player_5_def_pos,v_player_6_id,v_player_6_name,v_player_6_def_pos,v_player_7_id,v_player_7_name,v_player_7_def_pos,v_player_8_id,v_player_8_name,v_player_8_def_pos,v_player_9_id,v_player_9_name,v_player_9_def_pos,h_player_1_id,h_player_1_name,h_player_1_def_pos,h_player_2_id,h_player_2_name,h_player_2_def_pos,h_player_3_id,h_player_3_name,h_player_3_def_pos,h_player_4_id,h_player_4_name,h_player_4_def_pos,h_player_5_id,h_player_5_name,h_player_5_def_pos,h_player_6_id,h_player_6_name,h_player_6_def_pos,h_player_7_id,h_player_7_name,h_player_7_def_pos,h_player_8_id,h_player_8_name,h_player_8_def_pos,h_player_9_id,h_player_9_name,h_player_9_def_pos,additional_info,acquisition_info,game_id
0,18710504,0,Thu,CL1,None,1,FW1,None,1,0,2,54.0,D,None,None,None,FOR01,200.0,120.0,000000000,010010000,30.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,None,6.0,1.0,None,-1.0,None,4.0,1.0,1.0,1.0,0.0,0.0,27.0,9.0,0.0,3.0,0.0,0.0,31.0,4.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,None,0.0,0.0,None,-1.0,None,3.0,1.0,0.0,0.0,0.0,0.0,27.0,3.0,3.0,1.0,1.0,0.0,boakj901,John Boake,None,None,None,None,None,None,None,None,None,None,paboc101,Charlie Pabor,lennb101,Bill Lennon,mathb101,Bobby Mathews,prata101,Al Pratt,None,None,None,None,prata101,Al Pratt,mathb101,Bobby Mathews,whitd102,Deacon White,2.0,kimbg101,Gene Kimball,4.0,paboc101,Charlie Pabor,7.0,allia101,Art Allison,8.0,white104,Elmer White,9.0,prata101,Al Pratt,1.0,sutte101,Ezra Sutton,5.0,carlj102,Jim Carleton,3.0,bassj101,John Bass,6.0,selmf101,Frank Sellman,5.0,mathb101,Bobby Mathews,1.0,foraj101,Jim Foran,3.0,goldw101,Wally Goldsmith,6.0,lennb101,Bill Lennon,2.0,caret101,Tom Carey,4.0,mince101,Ed Mincher,7.0,mcdej101,James McDermott,8.0,kellb105,Bill Kelly,9.0,None,Y,FW1187105040


In [47]:
# create game table
#drop_game = '''
#DROP TABLE IF EXISTS game;
#'''

#run_command(drop_game)

create_game = '''

CREATE TABLE IF NOT EXISTS game (
game_id TEXT PRIMARY KEY,
date TEXT ,
number_of_game INTEGER,
park_id TEXT,
length_outs INTEGER,
day NUMERIC,
completion TEXT,
forfeit TEXT,
protest TEXT,
attendance NUMERIC,
length_minutes NUMERIC,
additional_info TEXT,
acquisition_info TEXT);
'''
data_game = '''

INSERT OR IGNORE INTO game
SELECT 
    game_id,
    date TEXT,
    number_of_game,
    park_id,
    length_outs,
    day_night,
    completion,
    forefeit,
    protest,
    attendance,
    length_minutes,
    additional_info,
    acquisition_info
FROM game_log;
'''

show_game = '''
SELECT *
FROM game
LIMIT 5;
'''

In [48]:
run_command(create_game)
run_command(data_game)
run_query (show_game)

,game_id,date,number_of_game,park_id,length_outs,day,completion,forfeit,protest,attendance,length_minutes,additional_info,acquisition_info
0,FW1187105040,18710504,0,FOR01,54,D,None,None,None,200,120,None,Y
1,WS3187105050,18710505,0,WAS01,54,D,None,None,None,5000,145,HTBF,Y
2,RC1187105060,18710506,0,RCK01,54,D,None,None,None,1000,140,None,Y
3,CH1187105080,18710508,0,CHI01,54,D,None,None,None,5000,150,None,Y
4,TRO187105090,18710509,0,TRO01,54,D,None,None,None,3250,145,HTBF,Y


In [ ]:
look_game_log = '''
SELECT sql FROM sqlite_master
WHERE name = "game_log"
AND type = "table";
'''

run_query(look_game_log)

In [50]:
create_team_appearance = '''

CREATE TABLE IF NOT EXISTS team_appearance (
    team_id TEXT,
    game_id TEXT,
    home BOOLEAN,
    league_id TEXT,
    score INTEGER,
    line_score TEXT,
    at_bats INTEGER,
    hits INTEGER,
    doubles INTEGER,
    triples INTEGER,
    homeruns INTEGER,
    rbi INTEGER,
    sacrifice_hits INTEGER,
    sacrifice_flies INTEGER,
    hit_by_pitch INTEGER,
    walks INTEGER,
    intentional_walks INTEGER,
    strikeouts INTEGER,
    stolen_bases INTEGER,
    caught_stealing INTEGER,
    grounded_into_double INTEGER,
    first_catcher_interference INTEGER,
    left_on_base INTEGER,
    pitchers_used INTEGER,
    individual_earned_runs INTEGER,
    team_earned_runs INTEGER,
    wild_pitches INTEGER,
    balks INTEGER,
    putouts INTEGER,
    assists INTEGER,
    errors INTEGER,
    passed_balls INTEGER,
    double_plays INTEGER,
    triple_plays INTEGER,
    PRIMARY KEY (team_id, game_id)
    FOREIGN KEY(team_id) REFERENCES team(team_id)
    FOREIGN KEY(game_id) REFERENCES game(game_id)
    FOREIGN KEY(league_id) REFERENCES league(league_id));
'''

In [51]:
run_query(g_log)

,date,number_of_game,day_of_week,v_name,v_league,v_game_number,h_name,h_league,h_game_number,v_score,h_score,length_outs,day_night,completion,forefeit,protest,park_id,attendance,length_minutes,v_line_score,h_line_score,v_at_bats,v_hits,v_doubles,v_triples,v_homeruns,v_rbi,v_sacrifice_hits,v_sacrifice_flies,v_hit_by_pitch,v_walks,v_intentional_walks,v_strikeouts,v_stolen_bases,v_caught_stealing,v_grounded_into_double,v_first_catcher_interference,v_left_on_base,v_pitchers_used,v_individual_earned_runs,v_team_earned_runs,v_wild_pitches,v_balks,v_putouts,v_assists,v_errors,v_passed_balls,v_double_plays,v_triple_plays,h_at_bats,h_hits,h_doubles,h_triples,h_homeruns,h_rbi,h_sacrifice_hits,h_sacrifice_flies,h_hit_by_pitch,h_walks,h_intentional_walks,h_strikeouts,h_stolen_bases,h_caught_stealing,h_grounded_into_double,h_first_catcher_interference,h_left_on_base,h_pitchers_used,h_individual_earned_runs,h_team_earned_runs,h_wild_pitches,h_balks,h_putouts,h_assists,h_errors,h_passed_balls,h_double_plays,h_triple_plays,hp_umpire_id,hp_umpire_name,1b_umpire_id,1b_umpire_name,2b_umpire_id,2b_umpire_name,3b_umpire_id,3b_umpire_name,lf_umpire_id,lf_umpire_name,rf_umpire_id,rf_umpire_name,v_manager_id,v_manager_name,h_manager_id,h_manager_name,winning_pitcher_id,winning_pitcher_name,losing_pitcher_id,losing_pitcher_name,saving_pitcher_id,saving_pitcher_name,winning_rbi_batter_id,winning_rbi_batter_id_name,v_starting_pitcher_id,v_starting_pitcher_name,h_starting_pitcher_id,h_starting_pitcher_name,v_player_1_id,v_player_1_name,v_player_1_def_pos,v_player_2_id,v_player_2_name,v_player_2_def_pos,v_player_3_id,v_player_3_name,v_player_3_def_pos,v_player_4_id,v_player_4_name,v_player_4_def_pos,v_player_5_id,v_player_5_name,v_player_5_def_pos,v_player_6_id,v_player_6_name,v_player_6_def_pos,v_player_7_id,v_player_7_name,v_player_7_def_pos,v_player_8_id,v_player_8_name,v_player_8_def_pos,v_player_9_id,v_player_9_name,v_player_9_def_pos,h_player_1_id,h_player_1_name,h_player_1_def_pos,h_player_2_id,h_player_2_name,h_player_2_def_pos,h_player_3_id,h_player_3_name,h_player_3_def_pos,h_player_4_id,h_player_4_name,h_player_4_def_pos,h_player_5_id,h_player_5_name,h_player_5_def_pos,h_player_6_id,h_player_6_name,h_player_6_def_pos,h_player_7_id,h_player_7_name,h_player_7_def_pos,h_player_8_id,h_player_8_name,h_player_8_def_pos,h_player_9_id,h_player_9_name,h_player_9_def_pos,additional_info,acquisition_info,game_id
0,18710504,0,Thu,CL1,None,1,FW1,None,1,0,2,54.0,D,None,None,None,FOR01,200.0,120.0,000000000,010010000,30.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,None,6.0,1.0,None,-1.0,None,4.0,1.0,1.0,1.0,0.0,0.0,27.0,9.0,0.0,3.0,0.0,0.0,31.0,4.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,None,0.0,0.0,None,-1.0,None,3.0,1.0,0.0,0.0,0.0,0.0,27.0,3.0,3.0,1.0,1.0,0.0,boakj901,John Boake,None,None,None,None,None,None,None,None,None,None,paboc101,Charlie Pabor,lennb101,Bill Lennon,mathb101,Bobby Mathews,prata101,Al Pratt,None,None,None,None,prata101,Al Pratt,mathb101,Bobby Mathews,whitd102,Deacon White,2.0,kimbg101,Gene Kimball,4.0,paboc101,Charlie Pabor,7.0,allia101,Art Allison,8.0,white104,Elmer White,9.0,prata101,Al Pratt,1.0,sutte101,Ezra Sutton,5.0,carlj102,Jim Carleton,3.0,bassj101,John Bass,6.0,selmf101,Frank Sellman,5.0,mathb101,Bobby Mathews,1.0,foraj101,Jim Foran,3.0,goldw101,Wally Goldsmith,6.0,lennb101,Bill Lennon,2.0,caret101,Tom Carey,4.0,mince101,Ed Mincher,7.0,mcdej101,James McDermott,8.0,kellb105,Bill Kelly,9.0,None,Y,FW1187105040


In [52]:
data_team_appearance = '''
INSERT OR IGNORE INTO team_appearance
SELECT
    h_name,
    game_id,
    1 AS home,
    h_league,
    h_score,
    h_line_score,
    h_at_bats,
    h_hits,
    h_doubles,
    h_triples,
    h_homeruns,
    h_rbi,
    h_sacrifice_hits,
    h_sacrifice_flies,
    h_hit_by_pitch,
    h_walks,
    h_intentional_walks,
    h_strikeouts,
    h_stolen_bases,
    h_caught_stealing,
    h_grounded_into_double,
    h_first_catcher_interference,
    h_left_on_base,
    h_pitchers_used,
    h_individual_earned_runs,
    h_team_earned_runs,
    h_wild_pitches,
    h_balks,
    h_putouts,
    h_assists,
    h_errors,
    h_passed_balls,
    h_double_plays,
    h_triple_plays
FROM game_log
    
    UNION

SELECT  
    v_name,
    game_id,
    0 as home,
    v_league,
    v_score,
    v_line_score,
    v_at_bats,
    v_hits,
    v_doubles,
    v_triples,
    v_homeruns,
    v_rbi,
    v_sacrifice_hits,
    v_sacrifice_flies,
    v_hit_by_pitch,
    v_walks,
    v_intentional_walks,
    v_strikeouts,
    v_stolen_bases,
    v_caught_stealing,
    v_grounded_into_double,
    v_first_catcher_interference,
    v_left_on_base,
    v_pitchers_used,
    v_individual_earned_runs,
    v_team_earned_runs,
    v_wild_pitches,
    v_balks,
    v_putouts,
    v_assists,
    v_errors,
    v_passed_balls,
    v_double_plays,
    v_triple_plays
FROM game_log;
'''

In [55]:
show_team_appearance = '''

SELECT *
FROM team_appearance
LIMIT 5;
'''

In [56]:
#run_command(create_team_appearance)
#run_command(data_team_appearance)
run_query (show_team_appearance)

,team_id,game_id,home,league_id,score,line_score,at_bats,hits,doubles,triples,homeruns,rbi,sacrifice_hits,sacrifice_flies,hit_by_pitch,walks,intentional_walks,strikeouts,stolen_bases,caught_stealing,grounded_into_double,first_catcher_interference,left_on_base,pitchers_used,individual_earned_runs,team_earned_runs,wild_pitches,balks,putouts,assists,errors,passed_balls,double_plays,triple_plays
0,ALT,ALT188404300,1,UA,2,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,ALT,ALT188405020,1,UA,3,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,ALT,ALT188405030,1,UA,5,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,ALT,ALT188405050,1,UA,2,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,ALT,ALT188405100,1,UA,9,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [7]:
!rm mlb.db

In [12]:
look_team_appearance = '''
SELECT sql FROM sqlite_master
WHERE name = "team_appearance"
AND type = "table";
'''

run_query(look_team_appearance)

,sql
0,"CREATE TABLE team_appearance (\n team_id TEXT,\n game_id TEXT,\n home BOOLEAN,\n league_id TEXT,\n score INTEGER,\n line_score TEXT,\n at_bats INTEGER,\n hits INTEGER,\n doubles INTEGER,\n triples INTEGER,\n homeruns INTEGER,\n rbi INTEGER,\n sacrifice_hits INTEGER,\n sacrifice_flies INTEGER,\n hit_by_pitch INTEGER,\n walks INTEGER,\n intentional_walks INTEGER,\n strikeouts INTEGER,\n stolen_bases INTEGER,\n caught_stealing INTEGER,\n grounded_into_double INTEGER,\n first_catcher_interference INTEGER,\n left_on_base INTEGER,\n pitchers_used INTEGER,\n individual_earned_runs INTEGER,\n team_earned_runs INTEGER,\n wild_pitches INTEGER,\n balks INTEGER,\n putouts INTEGER,\n assists INTEGER,\n errors INTEGER,\n passed_balls INTEGER,\n double_plays INTEGER,\n triple_plays INTEGER,\n PRIMARY KEY (team_id, game_id)\n FOREIGN KEY(team_id) REFERENCES team(team_id)\n FOREIGN KEY(game_id) REFERENCES game(game_id)\n FOREIGN KEY(league_id) REFERENCES league(league_id))"


In [57]:
create_person_appearance = '''

CREATE TABLE IF NOT EXISTS person_appearance (
    appearance_id TEXT,
    person_id TEXT,
    team_id TEXT,
    appearance_type_id TEXT;
'''

In [ ]:
data_person_appearance = '''
INSERT INTO person_appearance (
    game_id,
    team_id,
    person_id,
    appearance_type_id
)
    SELECT
        game_id,
        NULL,
        lf_umpire_id,
        "ULF"
    FROM game_log
    WHERE lf_umpire_id IS NOT NULL

UNION

    SELECT
        game_id,
        NULL,
        rf_umpire_id,
        "URF"
    FROM game_log
    WHERE rf_umpire_id IS NOT NULL

UNION

    SELECT
        game_id,
        v_name,
        v_manager_id,
        "MM"
    FROM game_log
    WHERE v_manager_id IS NOT NULL

UNION

    SELECT
        game_id,
        h_name,
        h_manager_id,
        "MM"
    FROM game_log
    WHERE h_manager_id IS NOT NULL

UNION

    SELECT
        game_id,
        CASE
            WHEN h_score > v_score THEN h_name
            ELSE v_name
            END,
        winning_pitcher_id,
        "AWP"
    FROM game_log
    WHERE winning_pitcher_id IS NOT NULL

UNION;
'''